In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, eb1fbc9b-ab6a-45f8-84c0-73fae606701a, 3, Finished, Available, Finished)

In [2]:
%%sql
create table if not exists Lakehouse_Demo.Dim_Customer
(
Customer_ID	string	,
Customer_Name	string	,
Segment	string	,
City	string	,
State	string	,
Country	string	,
Region	string	,
Created_TS	timestamp	,
Modified_TS	timestamp	  

)
using delta

StatementMeta(, eb1fbc9b-ab6a-45f8-84c0-73fae606701a, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
Max_Date = spark.sql("""select coalesce(max(Modified_TS),"1900-01-01") from Lakehouse_Demo.Dim_Customer""").first()[0]
print(Max_Date)

StatementMeta(, eb1fbc9b-ab6a-45f8-84c0-73fae606701a, 5, Finished, Available, Finished)

1900-01-01


In [4]:
df_Sales = spark.read.table("Lakehouse_Demo.Sales_Stg")

df_final = df_Sales.selectExpr("Customer_ID","Customer_Name","Segment","City","State", \
                   "Country","Region")\
                   .where(col("Modified_TS")>Max_Date)\
                   .drop_duplicates()

StatementMeta(, eb1fbc9b-ab6a-45f8-84c0-73fae606701a, 6, Finished, Available, Finished)

In [5]:
df_final.createOrReplaceTempView("ViewCustomer")

StatementMeta(, eb1fbc9b-ab6a-45f8-84c0-73fae606701a, 7, Finished, Available, Finished)

In [6]:
%%sql
Merge into Lakehouse_Demo.Dim_Customer as dc
using ViewCustomer as vc
on dc.Customer_ID=vc.Customer_ID
when matched then 
update set
dc.Customer_Name	=	vc.Customer_Name	,
dc.Segment	=	vc.Segment	,
dc.City	=	vc.City	,
dc.State	=	vc.State	,
dc.Country	=	vc.Country	,
dc.Region	=	vc.Region	,
dc.Modified_TS	=	current_timestamp()	

when not matched then insert
(
dc.Customer_ID	,
dc.Customer_Name	,
dc.Segment	,
dc.City	,
dc.State	,
dc.Country	,
dc.Region	,
dc.Created_TS	,
dc.Modified_TS	
)
values
(
vc.Customer_ID	,
vc.Customer_Name	,
vc.Segment	,
vc.City	,
vc.State	,
vc.Country	,
vc.Region	,
current_timestamp(),
current_timestamp
)

StatementMeta(, eb1fbc9b-ab6a-45f8-84c0-73fae606701a, 8, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [7]:
%%sql
select * from Lakehouse_Demo.Dim_Customer

StatementMeta(, eb1fbc9b-ab6a-45f8-84c0-73fae606701a, 9, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 9 fields>